In [1]:
%%HTML
<style>
.container { width:100% !important; }
.input{ width:50% !important;
       align: center;
    
      }
.text_cell{ width:70% !important;
            font-size: 16px;
          text-align: center;}
.title {align:center !important;}
</style>

In [2]:
import pandas as pd
import numpy as np
#Kung fu
from shaolin.core.dashboard import Dashboard
from shaolin.dashboards.slicers import ArraySlicer, AxisPlot, DataFrameSlicer, PanelSlicer
from shaolin.dashboards.plot_mappers import PlotMapper
from shaolin.dashboards.colormap import ColormapPicker
#Plotting with Vpython and Plotly
import vpython as vp
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.core.display import clear_output
init_notebook_mode(connected=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
juno_journey = pd.read_pickle('juno_journey.pck')

In [8]:
oplt = OrbitPlotter(juno_journey)
oplt.widget

In [18]:
oplt.sun.radius=1e7*3.5

In [9]:
oplt.init_display()
oplt.init_balls()

<IPython.core.display.Javascript object>

In [ ]:
ds = OrbitPlotterPlotly(juno_journey)
ds.widget

In [3]:
class DynamicSlider(Dashboard):
    
    def __init__(self,data,strf="%D",mapper_dict=None, **kwargs):
        if mapper_dict is None:
            mapper_dict = {'x':{'max':100000.0,
                        'min':0.0,
                        'step':0.1,
                        'high':10000.,
                        'low':0.,
                        'default':np.nan,
                        'map_data':True,
                        'fixed_active':True,
                       },
                   'y':{'max':100000.0,
                        'min':0.0,
                        'step':0.1,
                        'high':10000.,
                        'low':0.,
                        'default':np.nan,
                        'map_data':True,
                        'fixed_active':True,
                       },
                    'z':{'max':100000.0,
                        'min':0.0,
                        'step':0.1,
                        'high':10000.,
                        'low':0.,
                        'default':np.nan,
                        'map_data':True,
                        'fixed_active':True,
                       },
                   'size':{'max':100000,
                        'min':1,
                        'step':0.5,
                        'high':1000,
                        'low':100,
                        'default':120,
                        'map_data':True,
                        'fixed_active':False,
                       },

                   'fill_alpha':{'max':1.0,
                        'min':0.,
                        'step':0.05,
                        'high':0.95,
                        'low':0.3,
                        'default':1.,
                        'map_data':False,
                        'fixed_active':False,
                       },
                   'fill_color':{'default_color':'#FFFFFF','map_data':False,'step':0.05,'min':0.0,'low':0.0}
            }
        
        
        self._strf = strf
        #self._oc = oc
        self._data = None
        self.data = data
        mask = np.zeros(len(self.data.items.values))
        mask[0] = 1
        
        mapper = PlotMapper(self.data.ix[0,:,:].copy(),mapper_dict=mapper_dict, name='data_mapper',mode='interactive')
        axisplot = AxisPlot(self.data.items.values, mask=mask,name='axis_plot')
        slider_vals = [pd.to_datetime(x).strftime(strf) for x in self.data.items.values]
        #cols= oc.data.columns.tolist() + ['index']
        dash = ['c$n=orbit_slider',[mapper,axisplot,['r$N=slider_row',['(1,'+str(len(self.data.items.values))+',1,1)$d=Step',
                                  '@selslider$n=index_slider&o='+str(slider_vals),
                                  'btn$D=play','text$n=many'
                                 ]],['r$N=toggles_row',['@[True]$d=Slider&n=slider_btn']]
               ]]
        Dashboard.__init__(self,dash, **kwargs)
        self.observe(self.update)
        self.update()
        self.data_mapper.x.data_slicer.columns_slicer.sel_sli.value = 'rx'
        self.data_mapper.y.data_slicer.columns_slicer.sel_sli.value = 'ry'
        self.data_mapper.z.data_slicer.columns_slicer.sel_sli.value = 'rz'
        self.data_mapper.x.data_scaler.scale_chk.value = False
        self.data_mapper.y.data_scaler.scale_chk.value = False
        self.data_mapper.z.data_scaler.scale_chk.value = False
    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self,val):
        if isinstance(val,pd.DataFrame):
            old_ix = val.index
            val = val.reset_index()
            val.index = old_ix
            data = val.T.to_panel().copy()
            
        elif isinstance(val, pd.Panel):
            data = val.copy()
        else:
            print(type(val))
            raise NotImplementedError
        
        
        self._data = data
    def _update_layout(self, _=None):
        
        if self.slider_btn.value:
            self.axis_plot.axisplot.visible = True
            self.axis_plot.buttons_row.visible = True
        else:
            self.axis_plot.axisplot.visible = False
            self.axis_plot.buttons_row.visible = False
        self.axis_plot.update()

    def update(self, _=None):
        self._update_layout()
        self.index_slider.options = [pd.to_datetime(x).strftime(self._strf) for x in self.data.ix[::self.step.value,:,:].items.values]
        self.index = self.index_slider.options.index(self.index_slider.value)
        self.axis_plot.data = self.data.ix[::self.step.value,:,:].items.values
        self.axis_plot.mask = np.zeros(len(self.data.ix[::self.step.value,:,:].items.values))
        self.axis_plot.mask[self.index] = 1000
        self.data_mapper.data = self.data.ix[::self.step.value,:,:].ix[self.index,:,:]
        self.output = self.data_mapper.output

In [4]:
class OrbitPlotter(Dashboard):
    
    def __init__(self,data,**kwargs):
        self.data = data
        orbitslider = DynamicSlider(data,name='orbit_slider', mode='interactive')
        dash = ['c$N=orbit_plotter',['##Orbit plot$N=title',orbitslider]]
        Dashboard.__init__(self, dash, **kwargs)
        self.observe(self.update)
  
    def init_display(self, title="",width=1000,height=650,**kwargs):
        canvas = vp.canvas(title=title, width=width, height=height, **kwargs)
        return canvas
    
    @staticmethod
    def HTMLColorToRGB(colorstring):
        """ convert #RRGGBB to an (R, G, B) tuple """
        colorstring = colorstring.strip()
        if colorstring[0] == '#': colorstring = colorstring[1:]
        if len(colorstring) != 6:
            raise ValueError
        r, g, b = colorstring[:2], colorstring[2:4], colorstring[4:]
        r, g, b = [int(n, 16) for n in (r, g, b)]
        return (r, g, b)
    
    def init_balls(self):
        rgb = self.HTMLColorToRGB('#edd70b')
        color = vp.vector(rgb[0]/256.,rgb[1]/256.,rgb[2]/256.)
        self.sun = vp.sphere(pos=vp.vector(0,0,0),radius=1e7*3.5,color=color, make_trail=True)

        self.balls = {}
        for planet in self.orbit_slider.output.index:
            pos0 = vp.vector(self.orbit_slider.output.ix[planet,'x'],
                             self.orbit_slider.output.ix[planet,'y'],
                             self.orbit_slider.output.ix[planet,'z'])
            rgb = self.HTMLColorToRGB(self.orbit_slider.output.ix[planet,'fill_color'])
            color = vp.vector(rgb[0]/256.,rgb[1]/256.,rgb[2]/256.)
            self.balls[planet] = vp.sphere(pos=pos0,radius= self.orbit_slider.output.ix[planet,'size']*1000,color=color, make_trail=True)
        
    
    def update(self, _=None):
        for planet in self.orbit_slider.output.index:
            pos0 = vp.vector(self.orbit_slider.output.ix[planet,'x'],
                             self.orbit_slider.output.ix[planet,'y'],
                             self.orbit_slider.output.ix[planet,'z'])
            self.balls[planet].pos = pos0
            rgb = self.HTMLColorToRGB(self.orbit_slider.output.ix[planet,'fill_color'])
            self.balls[planet].color  = vp.vector(rgb[0]/256.,rgb[1]/256.,rgb[2]/256.)
            self.balls[planet].trail_color = vp.vector(rgb[0]/256.,rgb[1]/256.,rgb[2]/256.)
            self.balls[planet].radius = self.orbit_slider.output.ix[planet,'size']*1e6

In [5]:
class OrbitPlotterPlotly(Dashboard):
    
    def __init__(self,data,layout=None,**kwargs):
        if layout is None:
                layout = {'height': 800,
                          'margin': {'autoexpand': True, 'b': 80, 'l': 80, 'pad': 0, 'r': 80, 't': 100},
                          'plot_bgcolor':'#000',
                          'paper_bgcolor':'#000',
                          'font': {'color': '#fff', 'family': 'Verdana', 'size': 12},
                          'titlefont': {'color': '#fff', 'family': 'Verdana', 'size': 22},
                          'title':"Juno's journey",
                          'width': 1200,}
        self.ply_layout = go.Layout(**layout)
        
        mapper_dict_3d = {'x':{'max':100.0,
                    'min':0.0,
                    'step':0.1,
                    'high':1.,
                    'low':0.,
                    'default':np.nan,
                    'map_data':True,
                    'fixed_active':True,
                   },
               'y':{'max':100.0,
                    'min':0.0,
                    'step':0.1,
                    'high':1.,
                    'low':0.,
                    'default':np.nan,
                    'map_data':True,
                    'fixed_active':True,
                   },
                  'z':{'max':100.0,
                    'min':0.0,
                    'step':0.1,
                    'high':1.,
                    'low':0.,
                    'default':np.nan,
                    'map_data':True,
                    'fixed_active':True,
                   },
               'size':{'max':100000,
                    'min':1,
                    'step':0.5,
                    'high':20,
                    'low':10,
                    'default':12,
                    'map_data':False,
                    'fixed_active':False,
                   },
               'line_width':{'max':50,
                    'min':0,
                    'step':0.5,
                    'high':5,
                    'low':1,
                    'default':2,
                    'map_data':False,
                    'fixed_active':False,
                   },
               'fill_alpha':{'max':1.0,
                    'min':0.,
                    'step':0.05,
                    'high':0.95,
                    'low':0.3,
                    'default':1.,
                    'map_data':False,
                    'fixed_active':False,
                   },
               'line_alpha':{'max':1.0,
                    'min':0.,
                    'step':0.05,
                    'high':0.95,
                    'low':0.3,
                    'default':1.,
                    'map_data':False,
                    'fixed_active':False,
                   },
               'line_color':{'default_color':'black','map_data':False,'step':0.05,'min':0.0,'low':0.0},
               'fill_color':{'default_color':'blue','map_data':False,'step':0.05,'min':0.0,'low':0.0}
              }
        
        orbitslider = DynamicSlider(data,name='orbit_slider', mode='interactive', mapper_dict=mapper_dict_3d)
        dash = ['c$N=orbit_plotter',['##Orbit plot$N=title',orbitslider,'@sel$d=Marker mode&o=["lines+markers","lines","markers"]',"@[True]$d=Sun",
                                         ['c$N=slider_col',["@(1,100,1,12)$d=Sun size",'@(0.01,100.,1.,2.)$d=Marker size']]]]
        Dashboard.__init__(self, dash, **kwargs)
        self.observe(self.update)
        
        self.marker_mode.observe(self.update)
        self.data = data
        self.marker_mode.widget.height='5em'
        self.marker_mode.target.height='5em'
        
    @property
    def data(self):
        return self._data

    @data.setter
    def data(self,val):
        self._data = val
        self.update()


    def create_trace_scatter3d(self, data,name,color):
        x2, y2, z2 =  data[name]['rx'], data[name]['ry'], data[name]['rz']
        trace = go.Scatter3d(
            x=x2,
            y=y2,
            z=z2,
            mode=self.marker_mode.value,
            name=name,
            text=data[name]['time'],
            marker=dict(
                color=color,
                size=self.marker_size.value,
                symbol='circle',
                line=dict(
                    color=color,
                    width=3
                ),
                opacity=0.9
            )
        )
        return trace

    def planet_trace(self, _=None):

        # text_data = self.orbit_slider.data_mapper.data.copy().drop('index')
        #text_data['text'] = ''

        clear_output(wait=True)
        dft =  self.orbit_slider.output.copy()
        trace1 = go.Scatter3d(
            x=dft.x.values,
            y=dft.y.values,
            z=dft.z.values,
            mode='markers',
            #text=text_data['text'],
            marker=dict(
                size=dft['size'].values,
                color=dft['fill_color'].values,

                line=dict(
                    color=dft['line_color'].values,
                    width= dft['line_width'].values
                ),
                opacity=dft['fill_alpha'].values,
            ))

        return trace1

    def update(self, _=None):
        colors =  self.orbit_slider.data_mapper.line_color.cm_picker.map_data(range(len(self.data.columns.levels[0])),hex=True)
        #cp.map_data(range(6),hex=True)

        data = [self.create_trace_scatter3d(self.data.dropna(),name,colors[i]) for i,name in enumerate(self.data.columns.levels[0])]
        points = [self.planet_trace()]
        if self.sun.value:
                sun = go.Scatter3d(
                    x=[0.],
                    y=[0.],
                    z=[0.],
                    mode='Markers',
                    name='Sun',
                    text='Sun',
                    marker=dict(
                        color='#edd70b',
                        size=self.sun_size.value,
                        symbol='circle',
                        line=dict(
                            color='#edd70b',
                            width=1
                        ),
                        opacity=0.9
                    )
                )
                data.append(sun)

        #orbit_data = [create_trace_scatter3d(t.data,name,colors[i]) for i,name in enumerate(t.data.columns.levels[0])]
        #clusters = [create_trace_clusters(t.data,name) for name in ['first_','man_']]

        fig = go.Figure(data=data+points, layout=self.ply_layout)
        clear_output(wait=True)
        iplot(fig, filename='simple-3d-scatter')